# Stage 5 (1 1_stage_models_for_2_stage_train)

# Импортируем библиотеки

In [1]:
from typing import Dict, Any
import warnings


# ----------------
# Data processing
# ----------------
import dill

import numpy as np
import pandas as pd
import polars as pl

from tqdm.auto import tqdm

# ---------------------
# RecSys models imports
# ---------------------
from lightfm import LightFM

from rectools import Columns
from rectools.dataset import Dataset as RTDataset
from rectools.models import (
    PopularModel,
    LightFMWrapperModel,
    implicit_knn,
)
from implicit import nearest_neighbours
from mab2rec import BanditRecommender, LearningPolicy


# -------------------
# Metrics Evaluation
# -------------------
from metrics import RecommenderMetrics


warnings.filterwarnings("ignore")

RANDOM_STATE = 42

# Load Data

In [2]:
data_path = "../data/closed/"

In [3]:
base_models_data = pl.scan_parquet(data_path + "base_models_data.parquet")
ranker_data = pl.scan_parquet(data_path + "ranker_data.parquet")
test_df = pl.scan_parquet(data_path + "test_df.parquet")

base_models_data.schema, ranker_data.schema, test_df.schema

(Schema([('user_id', Int64),
         ('item_id', Int64),
         ('dt', Datetime(time_unit='ns', time_zone=None)),
         ('ui_inter', UInt32),
         ('u_total_inter', UInt32),
         ('weight', Float64),
         ('ui_entry', Int64),
         ('cum_weight', Float64)]),
 Schema([('user_id', Int64),
         ('item_id', Int64),
         ('dt', Datetime(time_unit='ns', time_zone=None)),
         ('ui_inter', UInt32),
         ('u_total_inter', UInt32),
         ('weight', Float64),
         ('ui_entry', Int64),
         ('cum_weight', Float64)]),
 Schema([('user_id', Int64), ('item_id', List(Int64))]))

# Обучение моделей первого уровня для обучения ранкера

In [4]:
models_path = "../data/models/"

Модели максимально простые, основанные на взаимодействиях пользователей и айтемов.

Фитчи айтемов оставим для переранжирования.

## Rectools Dataset

In [5]:
base_models_data.schema

Schema([('user_id', Int64),
        ('item_id', Int64),
        ('dt', Datetime(time_unit='ns', time_zone=None)),
        ('ui_inter', UInt32),
        ('u_total_inter', UInt32),
        ('weight', Float64),
        ('ui_entry', Int64),
        ('cum_weight', Float64)])

In [6]:
# Создадим датасет взаимодействий
current_dataset = RTDataset.construct(
    # Изменим датасет `base_models_data` под стандарт `rectools`
    # Оставим только нужные колонки и переименуем 
    interactions_df=base_models_data.select(
        [
            "user_id",
            "item_id",
            "dt",
            "cum_weight",
        ]
    )
    .rename(
        {
            "user_id": Columns.User,
            "item_id": Columns.Item,
            "dt": Columns.Datetime,
            "cum_weight": Columns.Weight,
        }
    )
    .collect()
    # преобразуем в формат pandas
    .to_pandas(),
)

## Качество работы моделей будем оценивать следующим образом

In [11]:
# Проверим качество на тестовой выборке
# Берем только пользователей, которые присутствуют
# в base и test выборках
with open(data_path + "b2t_users.dill", "rb") as f:
    b2t_users = dill.load(f)
b2t_users, b2t_users.shape

(array([4194304, 2097153, 3145730, ..., 3145696, 3145702, 2097143]), (444430,))

In [12]:
# Пользователей много, так что выберем
# 100 тысяч пользователей, на которых расчитаем метрики
b2t_users = np.random.choice(
    b2t_users,
    size=10**5,
    replace=False,
)
b2t_users, b2t_users.shape

(array([2431814, 4021511,  803540, ..., 1319516, 3885228, 4238440]), (100000,))

Выделим часть таблицы, на которой будем сверяться

In [13]:
metrics_df_tmp = test_df.filter(pl.col("user_id").is_in(b2t_users)).collect()

## Rectools PopularModel

In [10]:
popular_model = PopularModel()

In [11]:
popular_model.fit(current_dataset)

%clear

In [12]:
# Save model
with open(models_path + "popular_model.dill", "wb") as f:
    dill.dump(popular_model, f)

In [13]:
# Load model
with open(models_path + "popular_model.dill", "rb") as f:
    popular_model = dill.load(f)

## Rectools Implicit


In [7]:
knn_impl_cos_k50 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.CosineRecommender(K=50)
)

knn_impl_bm25_k50 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.BM25Recommender(K=50)
)

knn_impl_tfidf_k50 = implicit_knn.ImplicitItemKNNWrapperModel(
    model=nearest_neighbours.TFIDFRecommender(K=50)
)

In [8]:
# Fit models
knn_impl_cos_k50.fit(current_dataset)
knn_impl_bm25_k50.fit(current_dataset)
knn_impl_tfidf_k50.fit(current_dataset)

%clear

39 s for 3 models for train

~ 13 s for 1 model

In [9]:
# Save models

with open(models_path + "knn_impl_cos_k50.dill", "wb") as f:
    dill.dump(knn_impl_cos_k50, f)

with open(models_path + "knn_impl_bm25_k50.dill", "wb") as f:
    dill.dump(knn_impl_bm25_k50, f)

with open(models_path + "knn_impl_tfidf_k50.dill", "wb") as f:
    dill.dump(knn_impl_tfidf_k50, f)

In [10]:
# Load models

with open(models_path + "knn_impl_cos_k50.dill", "rb") as f:
    knn_impl_cos_k50 = dill.load(f)

with open(models_path + "knn_impl_bm25_k50.dill", "rb") as f:
    knn_impl_bm25_k50 = dill.load(f)

with open(models_path + "knn_impl_tfidf_k50.dill", "rb") as f:
    knn_impl_tfidf_k50 = dill.load(f)

### Test Cosine

In [ ]:
candidates_cos: pl.DataFrame = pl.from_pandas(
    knn_impl_cos_k50.recommend(
        b2t_users,
        current_dataset,
        # выдаем 10 кандидатов
        k=10,
        # рекомендуем уже просмотренные товары
        filter_viewed=False,
    )
).rename(
    {
        "score": "cos_score",
        "rank": "cos_rank",
    }
)

11 s for 100K users

In [ ]:
predictions = (
    candidates_cos.filter(pl.col("cos_rank") <= 10)
    .select(["user_id", "item_id"])
    .group_by("user_id")
    .agg(pl.col("item_id").alias("cos_recs"))
)

#### Calculate metrics

In [13]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp.join(
        other=predictions,
        how="left",
        on="user_id",
    ),
    model_preds_col="cos_recs",
)

{'ndcg@k': 0.29378310897228904,
 'recall@k': 0.19345844811084217,
 'map@k': 0.14826857938055873}

### Test BM25

In [ ]:
candidates_bm25: pl.DataFrame = pl.from_pandas(
    knn_impl_bm25_k50.recommend(
        b2t_users,
        current_dataset,
        # выдаем 10 кандидатов
        k=10,
        # рекомендуем уже просмотренные товары
        filter_viewed=False,
    )
).rename(
    {
        "score": "bm25_score",
        "rank": "bm25_rank",
    }
)

11 s for 100K users


In [ ]:
predictions = (
    candidates_bm25.filter(pl.col("bm25_rank") <= 10)
    .select(["user_id", "item_id"])
    .group_by("user_id")
    .agg(pl.col("item_id").alias("bm25_recs"))
)

#### Calculate metrics

In [17]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp.join(
        other=predictions,
        how="left",
        on="user_id",
    ),
    model_preds_col="bm25_recs",
)

{'ndcg@k': 0.2834074226497831,
 'recall@k': 0.18585607211950644,
 'map@k': 0.14008425056205992}

### Test TFIDF

In [ ]:
candidates_tfidf: pl.DataFrame = pl.from_pandas(
    knn_impl_tfidf_k50.recommend(
        b2t_users,
        current_dataset,
        # выдаем 10 кандидатов
        k=10,
        # рекомендуем уже просмотренные товары
        filter_viewed=False,
    )
).rename(
    {
        "score": "tfidf_score",
        "rank": "tfidf_rank",
    }
)

11 s for 100K users


In [ ]:
predictions = (
    candidates_tfidf.filter(pl.col("tfidf_rank") <= 10)
    .select(["user_id", "item_id"])
    .group_by("user_id")
    .agg(pl.col("item_id").alias("tfidf_recs"))
)

#### Calculate metrics

In [20]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp.join(
        other=predictions,
        how="left",
        on="user_id",
    ),
    model_preds_col="tfidf_recs",
)

{'ndcg@k': 0.28688514807266785,
 'recall@k': 0.18978574823916908,
 'map@k': 0.1462818330547252}

## Rectools LightFM

In [7]:
# Задаем модель
lfm_model = LightFMWrapperModel(
    LightFM(
        no_components=64,
        learning_rate=0.1,
        loss="warp",
        max_sampled=7,
    ),
    epochs=20,
    num_threads=6,
    verbose=1,
)

In [8]:
lfm_model.fit(dataset=current_dataset)
%clear

Epoch: 100%|██████████| 20/20 [06:53<00:00, 20.68s/it]

7 m

In [9]:
# Save model
with open(models_path + "lfm_model.dill", "wb") as f:
    dill.dump(lfm_model, f)

In [10]:
# Load model
with open(models_path + "lfm_model.dill", "rb") as f:
    lfm_model = dill.load(f)

### Test LightFM

In [14]:
candidates_lfm: pl.DataFrame = pl.from_pandas(
    lfm_model.recommend(
        b2t_users,
        current_dataset,
        # выдаем 10 кандидатов
        k=10,
        # рекомендуем уже просмотренные товары
        filter_viewed=False,
    )
).rename(
    {
        "score": "lfm_score",
        "rank": "lfm_rank",
    }
)

1m 10s for 100K users

In [15]:
predictions = (
    candidates_lfm.filter(pl.col("lfm_rank") <= 10)
    .select(["user_id", "item_id"])
    .group_by("user_id")
    .agg(pl.col("item_id").alias("lfm_recs"))
)

#### Calculate metrics

In [16]:
RecommenderMetrics.evaluate_recommender(
    metrics_df_tmp.join(
        other=predictions,
        how="left",
        on="user_id",
    ),
    model_preds_col="lfm_recs",
)

{'ndcg@k': 0.17871450371352096,
 'recall@k': 0.11630885515959145,
 'map@k': 0.0781008573648904}

## Получим рекомендации для обучения ранкера

In [11]:
candidates_data_path = models_path + "candidates_data/"

In [12]:
with open(data_path + "b2r_users.dill", "rb") as f:
    b2r_users = dill.load(f)
b2r_users, b2r_users.shape

(array([2097153,       5, 1048582, ..., 3145722, 1048574, 4194303]), (543921,))

### Cosine Recommender

In [11]:
# Cosine Recommender
with open(models_path + "knn_impl_cos_k50.dill", "rb") as f:
    pl.from_pandas(
        dill.load(f).recommend(
            b2r_users,
            current_dataset,
            # выдаем 15 кандидатов
            k=15,
            # рекомендуем уже просмотренные товары
            filter_viewed=False,
        )
    ).rename(
        {
            "score": "cos_score",
            "rank": "cos_rank",
        }
    ).write_parquet(
        candidates_data_path + "candidates_cos.parquet"
    )

58 s for 543K users

### BM25 Recommender


In [12]:
# BM25 Recommender
with open(models_path + "knn_impl_bm25_k50.dill", "rb") as f:
    pl.from_pandas(
        dill.load(f).recommend(
            b2r_users,
            current_dataset,
            # выдаем 15 кандидатов
            k=15,
            # рекомендуем уже просмотренные товары
            filter_viewed=False,
        )
    ).rename(
        {
            "score": "bm25_score",
            "rank": "bm25_rank",
        }
    ).write_parquet(
        candidates_data_path + "candidates_bm25.parquet"
    )

58 s for 543K users

### TFIDF Recommender


In [13]:
# TFIDF Recommender
with open(models_path + "knn_impl_tfidf_k50.dill", "rb") as f:
    pl.from_pandas(
        dill.load(f).recommend(
            b2r_users,
            current_dataset,
            # выдаем 15 кандидатов
            k=15,
            # рекомендуем уже просмотренные товары
            filter_viewed=False,
        )
    ).rename(
        {
            "score": "tfidf_score",
            "rank": "tfidf_rank",
        }
    ).write_parquet(
        candidates_data_path + "candidates_tfidf.parquet"
    )

58 s for 543K users

### LightFM Recommender


In [13]:
# LightFM Recommender
with open(models_path + "lfm_model.dill", "rb") as f:
    pl.from_pandas(
        dill.load(f).recommend(
            b2r_users,
            current_dataset,
            # выдаем 15 кандидатов
            k=15,
            # рекомендуем уже просмотренные товары
            filter_viewed=False,
        )
    ).rename(
        {
            "score": "lfm_score",
            "rank": "lfm_rank",
        }
    ).write_parquet(
        candidates_data_path + "candidates_lfm.parquet"
    )

5 m for 543K users

# Сливаем всех кандидатов в одну таблицу

In [21]:
models_path = "../data/models/"

In [22]:
candidates_data_path = models_path + "candidates_data/"

Так как LightFM умеет работать с warm и cold пользователями (PopularModel была взята для тех же целей), а PopularModel имеет **плохой score** относительно остальных моделей и **сильно увеличивает размерность** получаемого датасета с кандидатами, то от кандидатов PopularModel решено отказаться

In [23]:
default_values_merged = {
    "cos_score": pl.col("cos_score").min() - 0.01,
    "bm25_score": pl.col("bm25_score").min() - 0.01,
    "tfidf_score": pl.col("tfidf_score").min() - 0.01,
    "lfm_score": pl.col("lfm_score").min() - 0.01,
    "cos_rank": pl.col("cos_rank").max() + 1,
    "bm25_rank": pl.col("bm25_rank").max() + 1,
    "tfidf_rank": pl.col("tfidf_rank").max() + 1,
    "lfm_rank": pl.col("lfm_rank").max() + 1,
}

In [24]:
candidates_list = ["cos", "bm25", "tfidf", "lfm"]

candidates = pl.scan_parquet(
    candidates_data_path + f"candidates_{candidates_list[0]}.parquet"
).filter(pl.col(f"{candidates_list[0]}_rank") < 15)

for cand in candidates_list[1:]:
    candidates = (
        candidates.join(
            other=pl.scan_parquet(
                candidates_data_path + f"candidates_{cand}.parquet"
            ).filter(pl.col(f"{cand}_rank") < 15),
            how="outer",
            on=["user_id", "item_id"],
        )
        .with_columns(
            pl.col("user_id").fill_null(pl.col("user_id_right")),
            pl.col("item_id").fill_null(pl.col("item_id_right")),
        )
        .drop(["user_id_right", "item_id_right"])
    )


candidates.collect().with_columns(
    (
        pl.col(col_name).fill_null(default_values_merged[col_name])
        for col_name in default_values_merged.keys()
    )
).write_parquet(candidates_data_path + "candidates_full.parquet")

10 s for merge & save


|Stage|Description|Time (s)|Time (m)|
|---|---|---|---|
|**Data Prep & Training**||||
|Dataset Creation|Creation of dataset for level 1 models|4.2|0.07|
|KNN Training|Training 3 KNN models|40|0.67|
|LFM Training|Training LFM Model|420|7|
|Model Dumping|Saving trained models|3.5|0.06|
|**Inference**||||
|KNN Inference|3 KNN models (total) (543K records, 9.5k user/s per model)|174|2.9|
|LFM Inference|LFM Model (543K records, 2k user/s)|300|5|
|**Post Processing**||||
|Candidate Table Merge|Merging candidates into a single table|10|0.17|
|**Total Time**|**All Processes**|**951.7**|**15.85**|
